In [1]:
import pandas as pd
import os

In [3]:
df = pd.read_csv("../raw_data/ar_properties.csv")

In [5]:
df["id"] = range(0,len(df.id))

In [8]:
df.id

0              0
1              1
2              2
3              3
4              4
           ...  
999995    999995
999996    999996
999997    999997
999998    999998
999999    999999
Name: id, Length: 1000000, dtype: int64

In [6]:
df = df.rename(columns={
    "lat":"latitude",
    "lon":"longitude",
    "l1":"country", 
    "l2":"province", 
    "l3":"city",
    "l4":"district",
    "l5":"estate"
})

In [9]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'latitude',
       'longitude', 'country', 'province', 'city', 'district', 'estate', 'l6',
       'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered',
       'price', 'currency', 'price_period', 'title', 'description',
       'property_type', 'operation_type'],
      dtype='object')

In [10]:
df.l6.value_counts()

Series([], Name: count, dtype: int64)

In [11]:
df = df.drop(columns=["l6"])

In [12]:
df.ad_type.value_counts()

ad_type
Propiedad    1000000
Name: count, dtype: int64

In [13]:
df = df.drop(columns=["ad_type"])

In [14]:
df.operation_type.value_counts()

operation_type
Venta                782122
Alquiler             183927
Alquiler temporal     33951
Name: count, dtype: int64

In [15]:
df["operation_type"] = df["operation_type"].replace(regex={
    "Venta":"Sale",
    "Alquiler temporal":"Temporary Rent",
    "Alquiler":"Rent"})

In [16]:
df.operation_type.value_counts()

operation_type
Sale              782122
Rent              183927
Temporary Rent     33951
Name: count, dtype: int64

In [17]:
df.property_type.value_counts()

property_type
Departamento       373376
Otro               239720
Casa               190023
Lote                83792
PH                  35217
Local comercial     34493
Oficina             26365
Cochera              8908
Depósito             6502
Casa de campo        1604
Name: count, dtype: int64

In [18]:
df["property_type"] = df["property_type"].replace(regex={
    "Departamento":"Apartment",
    "Local comercial":"Commercial Premises",
    "Oficina":"Office",
    "Depósito":"Warehouse",
    "Otro": "Other",
    "Casa de campo":"Village House",
    "Casa":"House",
    "Cochera":"Garage",
    "Lote":"Terrain",
    "PH": "Horizontal Property"
    })

In [19]:
df.property_type.value_counts()

property_type
Apartment              373376
Other                  239720
House                  190023
Terrain                 83792
Horizontal Property     35217
Commercial Premises     34493
Office                  26365
Garage                   8908
Warehouse                6502
Village House            1604
Name: count, dtype: int64

In [20]:
cond_rent = df["operation_type"] == "Rent"
cond_sale = df["operation_type"] == "Sale"
cond_curr = df["currency"] == "USD"
df[cond_sale & cond_curr].price.mean(), \
    df[cond_rent & cond_curr].price.mean()

(231140.34107096842, 7379.891555198346)

In [21]:
df.price_period.value_counts()

price_period
Mensual    362978
Semanal        29
Diario          8
Name: count, dtype: int64

In [22]:
df["price_period"] = df["price_period"].replace(regex={
    "Mensual":"Monthly",
    "Semanal":"Weekly",
    "Diario":"Daily"})

In [23]:
df.price_period.value_counts()

price_period
Monthly    362978
Weekly         29
Daily           8
Name: count, dtype: int64

In [24]:
df.head()

,id,start_date,end_date,created_on,latitude,longitude,country,province,city,district,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,0,2020-12-25,9999-12-31,2020-12-25,-32.716652,-68.642692,Argentina,Mendoza,NaN,NaN,...,NaN,350.0,350.0,NaN,NaN,NaN,Excelentes Lotes Sobre Ruta 34,Corredor Responsable: VICTOR E. MONTIVERO - C....,Terrain,Sale
1,1,2020-12-25,9999-12-31,2020-12-25,-24.797723,-65.467514,Argentina,Salta,NaN,NaN,...,NaN,1541.0,1541.0,NaN,NaN,Monthly,TERRENO + VENTA + JARDINES DE SAN LORENZO +150...,Corredor Responsable: Pablo Castañeda - C.U.C....,Terrain,Sale
2,2,2020-12-25,2020-12-29,2020-12-25,-34.919373,-58.020591,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Monthly,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Terrain,Sale
3,3,2020-12-25,2020-12-29,2020-12-25,-34.919455,-58.024807,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Monthly,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Terrain,Sale
4,4,2020-12-25,9999-12-31,2020-12-25,-34.364924,-58.783143,Argentina,Bs.As. G.B.A. Zona Norte,Escobar,Belén de Escobar,...,NaN,18164.0,18164.0,NaN,NaN,Monthly,PANAMERICANA 47300,Nave principal 66 x 90 m: 6005 m2 cubiertos...,Other,Sale


In [25]:
df.country.value_counts()

country
Argentina         985725
Uruguay            13471
Estados Unidos       705
Brasil                99
Name: count, dtype: int64

In [26]:
columns_to_drop = df[df["country"] == "Estados Unidos"]
df = df.drop(columns_to_drop.index)

In [27]:
df.country.value_counts()

country
Argentina    985725
Uruguay       13471
Brasil           99
Name: count, dtype: int64

In [28]:
df.dtypes

id                   int64
start_date          object
end_date            object
created_on          object
latitude           float64
longitude          float64
country             object
province            object
city                object
district            object
estate              object
rooms              float64
bedrooms           float64
bathrooms          float64
surface_total      float64
surface_covered    float64
price              float64
currency            object
price_period        object
title               object
description         object
property_type       object
operation_type      object
dtype: object

In [29]:
# df["start_date"] = df["start_date"].astype("datetime64[s]")
# df["end_date"] = df["end_date"].astype("datetime64[s]")
# df["created_on"] = df["created_on"].astype("datetime64[s]")
# df["country"] = df["country"].astype("string")
# df["province"] = df["province"].astype("string")
# df["city"] = df["city"].astype("string")
# df["country"] = df["country"].astype("string")
# df["district"] = df["district"].astype("string")
# df["estate"] = df["estate"].astype("string")
# df["currency"] = df["currency"].astype("string")
# df["price_period"] = df["price_period"].astype("string")
# df["title"] = df["title"].astype("string")
# df["description"] = df["description"].astype("string")
# df["property_type"] = df["property_type"].astype("string")
# df["operation_type"] = df["operation_type"].astype("string")
df["id"] = df["id"].astype("int")

In [30]:
df.dtypes

id                   int64
start_date          object
end_date            object
created_on          object
latitude           float64
longitude          float64
country             object
province            object
city                object
district            object
estate              object
rooms              float64
bedrooms           float64
bathrooms          float64
surface_total      float64
surface_covered    float64
price              float64
currency            object
price_period        object
title               object
description         object
property_type       object
operation_type      object
dtype: object

In [31]:
df.head()

,id,start_date,end_date,created_on,latitude,longitude,country,province,city,district,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,0,2020-12-25,9999-12-31,2020-12-25,-32.716652,-68.642692,Argentina,Mendoza,NaN,NaN,...,NaN,350.0,350.0,NaN,NaN,NaN,Excelentes Lotes Sobre Ruta 34,Corredor Responsable: VICTOR E. MONTIVERO - C....,Terrain,Sale
1,1,2020-12-25,9999-12-31,2020-12-25,-24.797723,-65.467514,Argentina,Salta,NaN,NaN,...,NaN,1541.0,1541.0,NaN,NaN,Monthly,TERRENO + VENTA + JARDINES DE SAN LORENZO +150...,Corredor Responsable: Pablo Castañeda - C.U.C....,Terrain,Sale
2,2,2020-12-25,2020-12-29,2020-12-25,-34.919373,-58.020591,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Monthly,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Terrain,Sale
3,3,2020-12-25,2020-12-29,2020-12-25,-34.919455,-58.024807,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,...,NaN,1000.0,1000.0,NaN,NaN,Monthly,Lote en Venta de 1000 m2 en La Plata,Corredor Responsable: Rico Sebastián - Martill...,Terrain,Sale
4,4,2020-12-25,9999-12-31,2020-12-25,-34.364924,-58.783143,Argentina,Bs.As. G.B.A. Zona Norte,Escobar,Belén de Escobar,...,NaN,18164.0,18164.0,NaN,NaN,Monthly,PANAMERICANA 47300,Nave principal 66 x 90 m: 6005 m2 cubiertos...,Other,Sale


In [36]:
df = df.drop(labels="description", axis="columns")

In [39]:
if not os.path.exists("csv"):
    os.mkdir("csv")

In [40]:
for _, i in enumerate(range(0,1000001,5000)):
    df[i:i+5000].to_csv(f"csv/preprocessed_{_}.csv")